In [ ]:
import seagul.envs
import ray
import ray.rllib.agents.ars as ars
from ray.tune.logger import pretty_print
import tensorflow as tf
import time
from tqdm import trange
from ray import tune

from ray.tune.registry import register_env
from seagul.envs.mujoco.five_link import FiveLinkWalkerEnv

#env_name = "Walker2d-v3"
env_name = "five_link-v3"
###
def env_creator(env_config):
    return FiveLinkWalkerEnv()  # return an env instance

register_env("five_link-v3", env_creator)
###

ray.init()
config = ars.DEFAULT_CONFIG.copy()
config["num_gpus"] = 0
config["num_workers"] = 8
config["eager"] = False
config["model"]["fcnet_hiddens"] = []

trainer = ars.ARSTrainer(config=config, env=env_name)
# Can optionally call trainer.restore(path) to load a checkpoint.
for i in trange(500):
   # Perform one iteration of training the policy with ARS
    result = trainer.train()


    
checkpoint = trainer.save()
print("checkpoint saved at", checkpoint)
print(pretty_print(result))
ray.shutdown()

In [ ]:
import numpy as np
trainer.policy.compute(np.random.randn(13))

In [ ]:
import seagul.envs
import ray
import ray.rllib.agents.ars as ars
from ray.tune.logger import pretty_print
import tensorflow as tf


from ray.tune.registry import register_env
from seagul.envs.mujoco.five_link import FiveLinkWalkerEnv

def env_creator(env_config):
    return FiveLinkWalkerEnv()  # return an env instance

register_env("five_link-v3", env_creator)

ray.init()
config = ars.DEFAULT_CONFIG.copy()
config["num_gpus"] = 0
config["num_workers"] = 1
config["eager"] = False
trainer = ars.ARSTrainer(config=config, env="five_link-v3")

trainer.load("/Users/sgillen/ray_results/ARS_five_link-v3_2019-10-21_14-02-55lwgccrp3/checkpoint_10/checkpoint-10") 
trainer.restore("/Users/sgillen/ray_results/ARS_five_link-v3_2019-10-21_14-02-55lwgccrp3/checkpoint_10/checkpoint-10")


In [ ]:
import gym
import numpy as np
import matplotlib.pyplot as plt

env = gym.make(env_name)
obs = env.reset()

action_hist = []
state_hist  = []
reward_hist = []

done = False
while not done:
    actions = trainer.compute_action(obs.flatten())
    obs, reward, done, _ = env.step(np.asarray(actions))
    env.render()
    action_hist.append(np.copy(actions))
    state_hist.append(np.copy(obs))
    reward_hist.append(np.copy(reward))
    if done:
        break


print(sum(reward_hist))
plt.plot(action_hist)
plt.figure()
plt.plot(state_hist)